In [1]:
import os

import pandas as pd

/cephfs/projects/psoker/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


#### Load data and preprocess to common format

In [2]:
# First time data load.
ratings = pd.read_csv(
    os.path.join("..", "..", "data", "netflix", "combined_data_1.txt"),
    header=None,
    names=["userId", "rating", "timestamp"],
)
ratings2 = pd.read_csv(
    os.path.join("..", "..", "data", "netflix", "combined_data_2.txt"),
    header=None,
    names=["userId", "rating", "timestamp"],
)
ratings3 = pd.read_csv(
    os.path.join("..", "..", "data", "netflix", "combined_data_3.txt"),
    header=None,
    names=["userId", "rating", "timestamp"],
)
ratings4 = pd.read_csv(
    os.path.join("..", "..", "data", "netflix", "combined_data_4.txt"),
    header=None,
    names=["userId", "rating", "timestamp"],
)

# Append to the first dataframe
ratings = pd.concat([ratings, ratings2])
ratings = pd.concat([ratings, ratings3])
ratings = pd.concat([ratings, ratings4])
ratings.reset_index(drop=True, inplace=True)

# Del from memory
del ratings2, ratings3, ratings4

# Making movieId column
ratings["movieId"] = ratings[ratings["rating"].isna()]["userId"].str.replace(":", "")
ratings["movieId"].fillna(method="ffill", inplace=True)
ratings.dropna(subset="rating", inplace=True)

# Rearrange columns
ratings = ratings[["userId", "movieId", "rating", "timestamp"]]

# Modify formats
ratings["timestamp"] = pd.to_datetime(ratings["timestamp"]).astype(int)
ratings["rating"] = ratings["rating"].astype(float)
ratings["movieId"] = ratings["movieId"].astype(int)
ratings["userId"] = ratings["userId"].astype(int)

ratings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100480507 entries, 1 to 100498276
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 3.7 GB


In [3]:
# Save preprocessed dataset to csv file
ratings.to_parquet(os.path.join("..", "..", "data", "netflix", "ratings.parquet"), index=False)